# Three Parts:



1.   YOLO part for object detection ---> Output: Cropping Images
2.   EfficientNet part for classfication on cropped images. ---> Output: Food vector
3.   NER for recipe recommendation



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Part 1: YOLO

In [ ]:
# import libraries:

!pip install roboflow supervision opencv-python
import torch
from roboflow import Roboflow
import supervision as sv
import cv2
import os
from PIL import Image

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
from yolov5.models.yolo import Model
from yolov5.utils.dataloaders import LoadImagesAndLabels
from yolov5.utils.general import check_dataset


!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
!mv yolov5s.pt /content/yolov5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 29.57 MiB/s, done.
Resolving deltas: 100% (11714/11714), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 22.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yo

**NOW: upload the pretrained parameters best.pt from link:**

**Test on one image, let its name be: test.jpg**


In [ ]:
!yolo task=detect mode=predict model=/content/best.pt source=/content/test.jpg  save=True project=/content/results plots=True save_txt=True save_crop=True

Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv5s summary (fused): 193 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs

image 1/1 /content/test4.jpg: 640x480 12 dummys, 36.7ms
Speed: 3.1ms preprocess, 36.7ms inference, 598.8ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/results/predict4
1 label saved to /content/results/predict4/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict


**Now, the result are saved in /content/results folder**
**where the cropped image are inside: /content/results/predict/crops/dummy**

# Part2: EfficientNet

In [ ]:
# import libraries:

!pip install ultralytics torch torchvision matplotlib
!pip install optuna

import os
import zipfile
import shutil
import random
import torch
import optuna
import cv2
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
# NO need for this

# # Unzip the dataset
# import zipfile
# import os

# zip_path = "/content/new_dataset.zip"
# data_dir = "/content/"


# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(data_dir)

# print("Dataset unzipped successfully!")

Dataset unzipped successfully!


In [ ]:
# Step 2: Define Transformations and Load Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# train_dir = '/content/new_dataset/train'
# test_dir = '/content/new_dataset/valid'

# train_data = datasets.ImageFolder(train_dir, transform=transform)
# test_data = datasets.ImageFolder(test_dir, transform=transform)

# train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

#num_classes = len(train_data.classes)
classes =  ['apple', 'asparagus', 'bacon', 'banana', 'basil', 'beans', 'beef', 'bell pepper', 'bitter gourd', 'blueberries', 'bok choy', 'bread', 'broccoli', 'butter', 'cabbage', 'carrot', 'cauliflower', 'cheese', 'chicken', 'chillies', 'chocolate', 'coriander', 'corn', 'cream', 'cucumber', 'egg', 'eggplant', 'eggs', 'fish', 'flour', 'garlic', 'ginger', 'green beans', 'green chilies', 'green pepper', 'ham', 'jam', 'juice', 'lemon', 'lettuce', 'lime', 'meat', 'milk', 'mushroom', 'olive', 'onion', 'orange', 'parsley', 'potato', 'pumpkin', 'red bell pepper', 'salami', 'sauce', 'sausage', 'shrimp', 'spinach', 'strawberry', 'sugar', 'sweet potato', 'tomato', 'watermelon', 'yellow bell pepper', 'yoghurt']

num_classes = len(classes)
print(f"Number of classes: {num_classes}")

# Step 3: Define EfficientNet Model
def load_efficientnet_model(num_classes):
    model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

Number of classes: 63


In [ ]:
# Step 4: Train EfficientNet Model with Optuna


# ONLY UNCOMMENT THIS IF YOU WANT TO TRAIN


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def evaluate_model_and_plot_random_correct_examples(model, loader, device, num_samples=3):
#     model.eval()
#     all_preds, all_labels = [], []
#     correct_examples = []

#     with torch.no_grad():
#         for images, labels in loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, preds = torch.max(outputs, 1)

#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#             for i in range(len(images)):
#                 if preds[i] == labels[i]:
#                     correct_examples.append((images[i].cpu(), labels[i].cpu(), preds[i].cpu()))

#     if len(correct_examples) > num_samples:
#         correct_examples = random.sample(correct_examples, num_samples)

#     if correct_examples:
#         fig, axes = plt.subplots(1, len(correct_examples), figsize=(15, 5))
#         for i, (image, label, pred) in enumerate(correct_examples):
#             image = image.permute(1, 2, 0)
#             image = image * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
#             image = image.numpy().clip(0, 1)
#             axes[i].imshow(image)
#             axes[i].axis('off')
#             axes[i].set_title(
#                 f'Label: {test_data.classes[label.item()]}\nPredicted: {test_data.classes[pred.item()]}',
#                 fontsize=12
#             )
#         plt.tight_layout()
#         plt.show()

#     accuracy = accuracy_score(all_labels, all_preds)
#     precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
#     recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
#     f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

#     return accuracy, precision, recall, f1

# def objective(trial):
#     learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
#     batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
#     epochs = trial.suggest_int("epochs", 5, 10)

#     train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
#     test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

#     model = load_efficientnet_model(num_classes).to(device)
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#     criterion = nn.CrossEntropyLoss()

#     for epoch in range(epochs):
#         model.train()
#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#     accuracy, _, _, _ = evaluate_model_and_plot_random_correct_examples(model, test_loader, device)
#     return accuracy

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=5)

# best_params = study.best_params
# print("Best hyperparameters:", best_params)

# batch_size = best_params["batch_size"]
# epochs = best_params["epochs"]
# learning_rate = best_params["learning_rate"]

# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# model = load_efficientnet_model(num_classes).to(device)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# for epoch in range(epochs):
#     print('epoch: ', epoch)
#     model.train()
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

# torch.save(model, "/content/efficientnet_trained.pth")
# print("EfficientNet model saved!")

# accuracy, precision, recall, f1 = evaluate_model_and_plot_random_correct_examples(model, test_loader, device)
# print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

**Now upload the trained efficientNet hyperparameter from link:**

**The name of this file is: efficientnet_trained.pth**

In [ ]:
# Load the saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("/content/efficientnet_trained.pth").to(device)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# classify cropped images:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from PIL import Image
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt



preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



test_dir = '/content/results/predict4/crops/dummy'


# # Initialize metrics
# correct = 0
# total = 0
# total_loss = 0
# criterion = torch.nn.CrossEntropyLoss()

# Prepare for plotting images
plot_images = []
# actual_labels = []
predicted_labels = []


# for root, dirs, files in os.walk('/content/dummy'):
#     if '.ipynb_checkpoints' in dirs:
#         dirs.remove('.ipynb_checkpoints')  # Skip the checkpoint directory
class_names = classes
food_vector = []
# Make predictions on test images
for img_file in os.listdir(test_dir):
    # category_path = os.path.join(test_dir, category)
    # if os.path.isdir(category_path):
        # for img_file in os.listdir(category_path):
            img_path = os.path.join(test_dir, img_file)
            print(img_path)
            image = Image.open(img_path)
            plt.imshow(image)
            # Convert image to RGB if it has an alpha channel
            if image.mode != 'RGB':
                image = image.convert('RGB')


            img_tensor = preprocess(image).unsqueeze(0).to(device)
            # true_label = food_class.index(category)


            with torch.no_grad():
              output = model(img_tensor)
              _, predicted_class = torch.max(output, 1)
              #plot_images.append(image)
              #predicted_labels.append(class_names[predicted_class.item()])
                # output = mobilenet(img_tensor)
                # predicted_index = output.argmax(dim=1).item()
                # predicted_label = food_class[predicted_index]

                # Calculate loss
                # loss = criterion(output, torch.tensor([true_label]).to(device))
                # total_loss += loss.item()

            # Update metrics
            # correct += (predicted_index == true_label)
            # total += 1

            # Store for plotting
            plot_images.append(np.array(image))
            # actual_labels.append(category)

            #print(f"Predicted Class: {class_names[predicted_class.item()]}")
            print(class_names[predicted_class.item()])
            food_vector.append(class_names[predicted_class.item()])
            predicted_labels.append(class_names[predicted_class.item()])

            #plt.imshow(image)
            #plt.title(f'Predicted: {class_names[predicted_class.item()]}')
            #plt.axis('off')
            #plt.show()

# Calculate accuracy and average loss
# accuracy = correct / total if total > 0 else 0
# average_loss = total_loss / total if total > 0 else 0
# error_rate = 1 - accuracy

# Print the metrics
# print(f'Accuracy: {accuracy:.4f}')
# print(f'Average Loss: {average_loss:.4f}')
# print(f'Error Rate: {error_rate:.4f}')

# Plotting images with actual and predicted labels

num_images = len(plot_images)
print(num_images)
print("len of images: ", len(plot_images))
print(type(plot_images[0]))
plt.figure(figsize=(15, num_images // 5 + 2))  # Adjust the figure height based on number of images
for i in range(num_images):
    plt.subplot(num_images // 5 + 1, 5, i + 1)  # Create a grid of subplots
    plt.imshow(plot_images[i])
    plt.title(f'Predicted: {predicted_labels[i]}')
    plt.axis('off')
    #plt.show()


#plt.tight_layout()
#plt.savefig('output predictions')
#plt.show()
#plot_images[0].shape

/content/results/predict4/crops/dummy/test46.jpg
eggs
/content/results/predict4/crops/dummy/test410.jpg
corn
/content/results/predict4/crops/dummy/test42.jpg
cabbage
/content/results/predict4/crops/dummy/test412.jpg
carrot
/content/results/predict4/crops/dummy/test49.jpg
beans
/content/results/predict4/crops/dummy/test43.jpg
egg
/content/results/predict4/crops/dummy/test44.jpg
beef
/content/results/predict4/crops/dummy/test47.jpg
bok choy
/content/results/predict4/crops/dummy/test45.jpg
carrot
/content/results/predict4/crops/dummy/test411.jpg
butter
/content/results/predict4/crops/dummy/test48.jpg
fish
/content/results/predict4/crops/dummy/test4.jpg
corn
12
len of images:  12
<class 'numpy.ndarray'>


In [ ]:
!cp -r /content/results /content/drive/MyDrive/ColabNotebooks/APS360_Results

In [ ]:
food_vector

['milk',
 'beef',
 'egg',
 'bread',
 'strawberry',
 'carrot',
 'yellow bell pepper',
 'cabbage',
 'bacon',
 'eggplant',
 'corn',
 'spinach']

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pes12017000148/food-ingredients-and-recipe-dataset-with-images")

print("Path to dataset files:", path)

100%|██████████| 206M/206M [00:10<00:00, 21.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pes12017000148/food-ingredients-and-recipe-dataset-with-images/versions/1


In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import time

### GPU statement
try:
    spacy.require_gpu()
    print("GPU is available in the training.")
except:
    print("GPU is not available, using CPU instead.")

# load the Spacy model
NER_SPACY = spacy.load("en_core_web_sm")

# the list of words in noun but not food ingredients
words_not_food_in_noun = {
              # "cup", "teaspoon", "tablespoons", "cups", "ounce", "ounces", "pound", "pounds", "tsp",
              # "tbsp", "medium", "stick", "slices", "extract", "oz", "temperature", "room", "pieces",
              # "purpose", "total", "g", "ml", "lengthwise", "crosswise", "pinch", "½", "¼", "¾",
              # "package", "sprigs", "sticks", "halves", "inch", "bunch", "bunches", "parts", "quality",
              # "batch", "thermometer", "slice", "grind", "attachment", "wheel", "superfine", "quarts",
              # "baking", "ribbons", "dash", "layers", "cube", "skewers", "sheets", "moons", "round",
              # "rind", "top", "bottom", "height", "width", "diameter", "bias", "flameproof",
              # "rolling", "pin", "center", "frying", "mill", "maker", "heaping", "¼", "sticks",
              # "tops", "ends", "weights", "kitchen", "tool", "processor", "ramekins", "thickness",
              # "molds", "sheet", "paddle", "strip", "segments", "blocks", "form", "stand",
              # "lengths", "handle", "springform", "sheets", "platter", "knives", "blender", "mandoline",
              # "skillet", "strainer", "cast", "iron", "jar", "cans", "quart", "pint", "ruler",
              # "bottle", "mortar", "pestle", "sticks", "notes", "step", "layer", "loaf",
              # "pan", "batch", "inch",  "foil", "container", "tins", "shells",
              # "mold", "serving", "blender", "processor", "attachment", "form", "box",
              # "bowl", "spatula", "mixer", "wire", "frame", "plate", "sheet",
              # "nut", "oven", "tray", "cutter", "grater", "pot", "wok", "dough", "log", "weight",
              # "temperature", "reading", "minutes", "hours", "tables", "range", "measuring", "bag",
              # "envelope", "mat", "sprinkling", "tin", "edges", "ball", "circle", "seal", "quart",
              # "syrup", "slab", "bars", "kitchen", "molds", "ribs", "base", "lump", "skewer",
              # "thickness", "degree", "ladle", "grains", "milliliters", "gallon", "basting", "stem",
              # "tray", "handful", "frames", "loops", "bag", "label"
              }

# food ingredient word extracting
def food_word_extracting(ingredients_list, batch_size=32):
    food_extracted_output = []
    total_batch = (len(ingredients_list) +batch_size-1) // batch_size  ### calculate the total batch sizes
    start_time = time.time()

    for batch_idx in range(0, len(ingredients_list), batch_size):
        ### seperate the total batch into [0,32],[32,64]....
        batch = ingredients_list[batch_idx:batch_idx + batch_size]
        batch_docs = list(NER_SPACY.pipe(batch))
        batch_output = []

        ### the important steps, we ultilize the Spacy noun words extracting ability to extract nouns and propns from the sentences.
        ### one slice of the cake ----> "cake" will be extracted
        for doc in batch_docs:
            food_ing_words = []
            for token in doc:
                ### if the token is in the noun and propn and not in the word_not_food_in_noun list
                if token.pos_ in {"NOUN", "PROPN"} and token.text.lower() not in words_not_food_in_noun:
                    food_ing_words.append(token.text)
            batch_output.append(food_ing_words)
        food_extracted_output.extend(batch_output)

        # print the logs
        time_cost = time.time() - start_time
        if (batch_idx // batch_size+1) % 100 == 0:  # print log every 10 batchs
            print(f"Processed batch {batch_idx // batch_size+1}/{total_batch}. Time cost is : {time_cost:.2f} seconds")
    total_time = time.time() - start_time
    print(f"Word extracting process using NER completed in {total_time:.2f} seconds.")
    return food_extracted_output

# data loading
data_path = "/root/.cache/kagglehub/datasets/pes12017000148/food-ingredients-and-recipe-dataset-with-images/versions/1"
csv_file = os.path.join(data_path, "Food Ingredients and Recipe Dataset with Image Name Mapping.csv")
df = pd.read_csv(csv_file)

# data cleaning
df = df.dropna(subset=['Cleaned_Ingredients', 'Title', 'Instructions'])
df = df.drop_duplicates(subset=['Title'])

### Standardize the cleaned_ingredients column (the improving step: we use the Spacy ner network to do the word extracting)
ingre_cleaned_merge = []
for cleaned_ingredients in df['Cleaned_Ingredients']:
    # use eval function and ' ' to merge every word into a sentence with the form of python list
    combined_text = " ".join(eval(cleaned_ingredients))
    ingre_cleaned_merge.append(combined_text)

# use food_extracting function to extract the food words in the merged sentences
ingredients_list = food_word_extracting(ingre_cleaned_merge)
df['Ingredients_list'] = ingredients_list

# input vector from YOLO output
YOLO_detected_ingredients = food_vector

# collect all ingredients from the ingredients list
ingredient_list_total = set()
for ingredients in df['Ingredients_list']:
    for ingredient in ingredients:
        if ingredient.strip():
            ingredient_list_total.add(ingredient)

# merge the YOLO food ingredient into the ingredient_list_total
ingredient_list_total.update(YOLO_detected_ingredients)

### use the ingredients as the labels of our MLB
mlb = MultiLabelBinarizer(classes=list(ingredient_list_total))
ingredient_vectors = mlb.fit_transform(df['Ingredients_list'])

# convert YOLO vector into the one hot encoding
YOLO_vector_one_hot = mlb.transform([YOLO_detected_ingredients])[0]

# calculate the cosine similarity
cos_sim = cosine_similarity([YOLO_vector_one_hot], ingredient_vectors).flatten()

## exclude the recipe with cosine similarity 0
non_zero_indices = cos_sim > 0
filtered_cos_sim = cos_sim[non_zero_indices]
filtered_indices = non_zero_indices.nonzero()[0]

# sort recipes without 0 cosine similarity and get the top 5 recipes
top_5_recipe_idx = filtered_cos_sim.argsort()[-5:][::-1]
top_5_recipe_idx_filtered = filtered_indices[top_5_recipe_idx]
top_5_recipes = df.iloc[top_5_recipe_idx_filtered]

### output adjust function, adjust the output display
def output_adjust(text, line_length=160):
    words = text.split()
    text_adjust = ""
    line = ""
    for word in words:
        if len(line) + len(word) + 1 > line_length:
            text_adjust += line + "\n"
            line = word
        else:
            if line:
                line += " " + word
            else:
                line = word
    text_adjust += line
    return text_adjust

# print the output
print("Top 5 Recommended Recipes based on the food detection in fridge:")
for i, index in enumerate(top_5_recipe_idx_filtered, start=1):
    title = df.iloc[index]['Title']
    instructions = df.iloc[index]['Instructions']
    ingredients = df.iloc[index]['Cleaned_Ingredients']
    similarity = cos_sim[index]
    formatted_instructions = output_adjust(instructions)
    ingredients = output_adjust(ingredients)
    print(f"Top{i}. Title: {title}, Cosine Similarity: {similarity:.4f}\n")
    print(f"Ingredients:\n{ingredients}\n")
    print(f"Instructions:\n{formatted_instructions}\n")


GPU is available in the training.
Processed batch 100/416. Time cost is : 27.97 seconds
Processed batch 200/416. Time cost is : 44.95 seconds
Processed batch 300/416. Time cost is : 58.21 seconds
Processed batch 400/416. Time cost is : 71.48 seconds
Word extracting process using NER completed in 73.19 seconds.
Top 5 Recommended Recipes based on the food detection in fridge:
Top1. Title: Teeny-Weeny Coxinha, Cosine Similarity: 0.2635

Ingredients:
['1 quart vegetable oil, for frying', '3 1/2 cups low-sodium chicken broth', '1 onion, peeled and quartered', '1 carrot, peeled and quartered', '1 celery rib,
quartered', '1 large chicken breast', '8 ounces packaged cream cheese, softened', '1 ear corn, kernels cut off the cob', '2 green onions, thinly sliced', '1
garlic clove, minced', 'Salt and pepper, to taste', '1/2 tablespoon extra-virgin olive oil', '2 cups all-purpose flour', '1 egg', '1 tablespoon whole milk', '1
cup plain Italian bread crumbs', 'Salt and pepper', 'to taste']

Instruct

In [ ]:
print(" Cleaned ingredient list[5] Using SPACY:")
ingredients = df['Ingredients_list'].iloc[5]
line_length = 9
for i in range(0, len(ingredients), line_length):
    print(ingredients[i:i + line_length])